In [1]:
import numpy as np
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Rede Neural - Convolucional

In [2]:
# Criando estrutura da rede neural
classificador = Sequential()
classificador.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2,2)))
classificador.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2,2)))
classificador.add(Flatten())
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 1, activation = 'sigmoid'))
# Compilando rede neural
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [3]:
# Resumo do modelo criado
classificador.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 29, 29, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0

In [5]:
# Usando as classes ImageDataGenerator e flow_from_directory do keras
gerador_treinamento = ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range = 7,
    horizontal_flip = True,
    shear_range = 0.2,
    height_shift_range = 0.07,
    zoom_range = 0.2
)

gerador_teste = ImageDataGenerator(rescale = 1. / 255)

base_treinamento = gerador_treinamento.flow_from_directory(
    '../dados/dataset/training_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

base_teste = gerador_teste.flow_from_directory(
    '../dados/dataset/test_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [6]:
base_treinamento.class_indices

{'cachorro': 0, 'gato': 1}

In [7]:
classificador.fit_generator(
    base_treinamento, 
    steps_per_epoch = 4000 / 32,
    epochs = 10, 
    validation_data = base_teste,
    validation_steps = 1000 / 32
)

/Users/houston/Repositorios/DeepLearningPython/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/10
125/125 [==============================] - 38s 295ms/step - loss: 0.8923 - accuracy: 0.5701 - val_loss: 0.7321 - val_accuracy: 0.5110
Epoch 2/10
125/125 [==============================] - 33s 266ms/step - loss: 0.6855 - accuracy: 0.6089 - val_loss: 1.1512 - val_accuracy: 0.4980
Epoch 3/10
125/125 [==============================] - 33s 267ms/step - loss: 0.6191 - accuracy: 0.6388 - val_loss: 0.7733 - val_accuracy: 0.5630
Epoch 4/10
125/125 [==============================] - 32s 252ms/step - loss: 0.6005 - accuracy: 0.6713 - val_loss: 0.6610 - val_accuracy: 0.6520
Epoch 5/10
125/125 [==============================] - 31s 249ms/step - loss: 0.5850 - accur

In [8]:
imagem_teste = image.load_img('./dataset/test_set/cachorro/dog.3500.jpg', target_size = (64,64))
imagem_teste = image.img_to_array(imagem_teste)
imagem_teste /= 255
imagem_teste = np.expand_dims(imagem_teste, axis = 0)

previsao = classificador.predict(imagem_teste)
previsao

array([[0.13562796]], dtype=float32)

In [9]:
previsao = (previsao > 0.5)
previsao

array([[False]])